In [2]:
%load_ext ipython_sparql_pandas
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper('http://DESKTOP-CELL0BF:7200/repositories/20qsmall')
sparql.setReturnFormat(JSON)

In [18]:
#from the query results extract a list that contains the counts after splitting on certain attributes
import re
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .', '?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def prepareProps(popEntities):
    Distribution = {}
    for i in popEntities:
        Distribution[i]= 1/len(popEntities)
    return Distribution


def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
    #print(listOfObj[0])
    return(listOfObj,listOfPred)


  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            group by ?p ?o 
            ORDER BY ABS( {left/2} - ?count )
            """) 
        
    return FilterQuestion

In [19]:
def numberleft(PosFilters,NegFilters):
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
            {PosFilters}{NegFilters} 
          }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))
    
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
            {PosFilters}
            {NegFilters} 
            }}
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    results = []
    for r in qres["results"]["bindings"]:
        results.append(r["s"]["value"])
    return results


In [20]:
# resets are variables
def game_reset():
    left = numberleft("","")
    FilterQuestion = updateQuery(left, "","")
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [32]:
def prepareProps(popEntities):
    Distribution = {}
    for i in popEntities:
        Distribution[i] = 1
        #Distribution[i]= 1/len(popEntities)
    return Distribution
    

def findPosAndNeg(listOfObj, listOfPred, index):
    Filter = '?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.'
    negFilter='FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}'
    EntitiesPositive = popentities(Filter, "")
    EntitiesNegative = popentities("", negFilter)
    return (EntitiesPositive, EntitiesNegative)
    
                       
def findSplits():
    possibleObj=[]
    possiblePred =[]
    count = []
    query =  f"""
            select (count(?s) as ?count)   ?o ?p
            {{               
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
        ?s ?p ?o.
            }}
        group by  ?o ?p
        ORDER BY desc ( ?count )
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        possibleObj.append(r["o"]["value"])
        possiblePred.append(r["p"]["value"])
        count.append(r["count"]["value"])
   
    return (possibleObj, possiblePred, count)

def determineBestSplit(possibleObj, possiblePred, count):
    bestSplit=-1
    indexBestSplit = -1
    """branches = range(2,250)
    for i in random.sample(branches, 30):""" 
    for i in range(350):
        #Fix The nummerical thing here::: (skip over i if nummer)
        if str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>') in AttrHistory:
            continue
        else:
            Yes = pos.get(str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>'))
            No = negs.get(str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>'))
            OccurenceYes = len(Yes)
            OccurenceNo=len(No)
            """Yes, No = findPosAndNeg(possibleObj, possiblePred, i)
            OccurenceYes=len(Yes)
            OccurenceNo=len(No)"""
            WeightYes=0
            WeightNo=0
            for y in Yes:
                WeightYes+=Distribution[y]
            for n in No:
                WeightNo+=Distribution[n]
            SumWeights=WeightYes+WeightNo
            ProbabilityYes=WeightYes/SumWeights
            ProbabilityNo=WeightNo/SumWeights
            Entropy = - (ProbabilityYes * math.log(ProbabilityYes+0.0001,2) + ProbabilityNo * math.log(ProbabilityNo+0.0001,2))
            #print(str(possiblePred[i]) +'> <'+str(possibleObj[i]), Entropy)
            if Entropy > bestSplit:
                bestSplit=Entropy
                indexBestSplit=i
    #print(bestSplit, possibleObj[indexBestSplit], possiblePred[indexBestSplit])
    return(indexBestSplit, bestSplit)

def WrongAnswerRate(answer, chance):
    if randint(0,100)<chance:
        print("haha fooled!")
        if answer:
            answer=False
        else:
            answer=True
    return(answer)
        
def wrongAnswerList( amount):
    l = []
    i = -1
    for i in range(20-amount):
        l.append(True)
    for i in range(amount):
        l.append(False)
    random.shuffle(l)
    return(l, i)

def retrieveAnswer(answer, l, i):
    i+=1
    if l[i]:
        return(answer, i)
    else:
        print("wrong answer incoming")
        return(not answer, i)
"""preparePops
enter loop
find splits, determine best split, generate question (update probabilities)"""
                             
                             
        


'preparePops\nenter loop\nfind splits, determine best split, generate question (update probabilities)'

In [22]:
def generateHumanQuestion(listOfObj, listOfPred, index, subject, Distribution):
    
    AttrHistory.append('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>')
    answer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]}?' )
    
    Yes, No = findPosAndNeg(listOfObj, listOfPred, index)
    if answer.startswith('y'):   
        for y in Yes:
            Distribution[y]= Distribution[y]*0.9
        for n in No:
            Distribution[n]= Distribution[n]*0.1
            
    else:
        for y in Yes:
            Distribution[y] = Distribution[y]*0.01
        for n in No:
            Distribution[n] = Distribution[n]*0.99
            
    print(Distribution[subject[1:-1]])
    return Distribution

def allSubjects(qres, ind):
    for r in qres["results"]["bindings"][ind:ind+1]:
        return("<"+r["s"]["value"]+">")

def loadSubjects():

    query = f"""
    select distinct ?s
    where {{
        ?s ?p ?o .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres
    
def pickSubject(qres):
    ind=randint(0,134)
    for r in qres["results"]["bindings"][ind:ind+1]:
        return("<"+r["s"]["value"]+">")

def findAnswer(subject,p,o):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        ?s ?p ?o .
            FILTER(?s = {subject})
            FILTER(?p = {p})
            FILTER(?o = {o})
            }}
            """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [23]:
def generateBotQuestionManipulate(listOfObj, listOfPred, index, subject, Distribution,itteration):
    AttrHistory.append('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>')
    print(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]}?' )
    answer = findAnswer(subject, '<' + str(listOfPred[index]) +'>', '<'+str(listOfObj[index])+ '>')
    """if itteration in [1,6,15,20]:
        answer = not answer"""
    #answer = WrongAnswerRate(answer, 10)

    answer, initilaizeCount = retrieveAnswer(answer, answerPattern,i-1)
    print(answer)
    Yes, No = findPosAndNeg(listOfObj, listOfPred, index)
    """OccurenceYes=len(Yes)
    OccurenceNo=len(No)
    NEntities=OccurenceYes+OccurenceNo
    ProbabilityYes=OccurenceYes/NEntities
    ProbabilityNo=OccurenceNo/NEntities"""

    if answer:   
        for y in Yes:
            Distribution[y]= Distribution[y]*0.99
        for n in No:
            Distribution[n]= Distribution[n]*0.01
            
    else:
        for y in Yes:
            Distribution[y] = Distribution[y]*0.01
        for n in No:
            Distribution[n] = Distribution[n]*0.99
            
    print(Distribution[subject[1:-1]])
    return Distribution

In [29]:
def loadAllAttributes(possibleObj,possiblePred):
    EntsPerAttribute={}
    EntsWithoutAttribute={}
    for i in range(350):
        Yes, No = findPosAndNeg(possibleObj, possiblePred, i)

        attr=str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>')
        EntsPerAttribute[attr]=Yes
        EntsWithoutAttribute[attr]=No
    return(EntsPerAttribute, EntsWithoutAttribute)


"""possibleObj, possiblePred, count = findSplits()
pos, negs= loadAllAttributes(possibleObj,possiblePred)"""

In [34]:
#without printing results
from random import randint
import random
import math
import time 
wincount = 0
gamesplayed = 0
wrongAnswersPerQuestion = 4
totalgames = 100
Entrop_over_game = []
list_losses=[]
list_wins=[]

possibleObj, possiblePred, count = findSplits()
loadAllAttributes(possibleObj,possiblePred)
qres = loadSubjects()

while gamesplayed < totalgames:
    answerPattern, initilaizeCount = wrongAnswerList(wrongAnswersPerQuestion)
    Entropies=[]
    AttrHistory= ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    # Resets all variables
    i=0
    outofguess = []
    # pick subject
    #subject = pickSubject(qres)
    subject = allSubjects(qres, gamesplayed)
    print("\n" , subject)
    popularEntities =  popentities("\n".join(PosAttr),"")
    Distribution = prepareProps(popularEntities)
    
    while i < 20:

        # list of Pred and Obj for questions
        #listOfObj ,listOfPred = extractCountAndAttr(qres)

        # if only 1 popular entity is left break


        # ask question and add filters to query
        print('Question: %s' %(i+1))

        indexSplit, Entropy=determineBestSplit(possibleObj, possiblePred, count)
        Distribution = generateBotQuestionManipulate(possibleObj, possiblePred, indexSplit,subject, Distribution, i)
        i+=1
        Entropies.append(Entropy)
    # Prints answer
    #print(Distribution)
    highProb=-1
    for j in Distribution.values():
        if j > highProb:
            highProb=j
    answer = list(Distribution.keys())[list(Distribution.values()).index(highProb)]
    print("The Answer is: ", answer, "With a Probability of: ", highProb)
    print(subject)
    if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
        wincount +=1
        list_wins.append(answerPattern)
    else:
        list_losses.append(answerPattern)
    gamesplayed+=1
    Entrop_over_game.append(Entropies)
    time.sleep(1)
    
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")


 <http://yago-knowledge.org/resource/David_Bowie>
Question: 1
Does the thing you are looking for have the attribute: hasOccupation Actor?
True
0.99
Question: 2
Does the thing you are looking for have the attribute: hasOccupation Film_producer?
False
0.9801
Question: 3
Does the thing you are looking for have the attribute: nationality United_States?
False
0.9702989999999999
Question: 4
Does the thing you are looking for have the attribute: nationality United_Kingdom?
True
0.96059601
Question: 5
Does the thing you are looking for have the attribute: award Order_of_the_British_Empire?
True
0.9509900498999999
Question: 6
Does the thing you are looking for have the attribute: hasOccupation Stage_actor?
wrong answer incoming
True
0.009509900499
Question: 7
Does the thing you are looking for have the attribute: award Academy_Award_for_Best_Supporting_Actress?
False
0.00941480149401
Question: 8
Does the thing you are looking for have the attribute: award Theatre_World_Award?
False
0.009320653

In [35]:
first=[]
first2=[]
first3=[]
first4=[]
first5.append(i[4])
first6.append(i[5])
first7.append(i[6])
first8.append(i[7])
first9.append(i[8])
first10.append(i[9])
first11.append(i[10])
first12.append(i[11])
first13.append(i[12])
first14.append(i[13])
first15.append(i[14])
first16.append(i[15])
first17.append(i[16])
first18.append(i[17])
first19.append(i[18])
first20.append(i[19])
for i in list_losses:
    first.append(i[0])
    first2.append(i[1])
    first3.append(i[2])
    first4.append(i[3])
    first5.append(i[4])
    first6.append(i[5])
    first7.append(i[6])
    first8.append(i[7])
    first9.append(i[8])
    first10.append(i[9])
    first11.append(i[10])
    first12.append(i[11])
    first13.append(i[12])
    first14.append(i[13])
    first15.append(i[14])
    first16.append(i[15])
    first17.append(i[16])
    first18.append(i[17])
    first19.append(i[18])
    first20.append(i[19])
print(first)

NameError: name 'first' is not defined

In [ ]:
for i in Entrop_over_game: 
    print(sum(i))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xax = numpy.array(range(0,134))
for i in dis_over_time:
    yax=[]
    for j in i:
        yax.append(j)
    plt.plot(xax, yax)
plt.show

color = plt.cm.rainbow(np.linspace(0, 1, len(dis_over_time)))
for i, c in zip(range(len(dis_over_time)), color):
    yax=dis_over_time[i]
    plt.plot(xax, yax, c=c)

In [ ]:
yax = []
xax = numpy.array(range(0,134))

for j in range(9):
    yax = []
    for i in analyse_dis[j].values():
        yax.append(i)
    plt.plot(xax, yax)
    plt.show()

In [ ]:
from random import randint
import random
import math
import time 

possibleObj, possiblePred, count = findSplits()
popularEntities =  popentities("\n".join(PosAttr),"")
Distribution = prepareProps(popularEntities)


bestSplit=-1
indexBestSplit = -1
#branches = range(2,250)
#for i in random.sample(branches, 30): 
for i in range(25):
    #Fix The nummerical thing here::: (skip over i if nummer)
    if str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>') in AttrHistory:
        continue
    else:
        Yes, No = findPosAndNeg(possibleObj, possiblePred, i)
        OccurenceYes=len(Yes)
        OccurenceNo=len(No)
        WeightYes=0
        WeightNo=0
        for y in Yes:
            WeightYes+=Distribution[y]
        for n in No:
            WeightNo+=Distribution[n]
        SumWeights=WeightYes+WeightNo
        ProbabilityYes=WeightYes/SumWeights
        ProbabilityNo=WeightNo/SumWeights
        Entropy = - (ProbabilityYes * math.log(ProbabilityYes+0.0001,2) + ProbabilityNo * math.log(ProbabilityNo+0.0001,2))
        print(Entropy, possiblePred[i], possibleObj[i])
        #print(str(possiblePred[i]) +'> <'+str(possibleObj[i]), Entropy)
        if Entropy > bestSplit:
            bestSplit=Entropy
            indexBestSplit=i
#print(bestSplit, possibleObj[indexBestSplit], possiblePred[indexBestSplit])
print(bestSplit, indexBestSplit)

In [33]:
list_wins

[[True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False,
  True],
 [True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False],
 [True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False],
 [True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 [True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True],
 [True,
  True,
